
COMPILADOR - ANÁLISE LÉXICA E SINTÁTICA LL(1) E SLR(1)
Implementação de um compilador para uma linguagem de programação simplificada



In [1]:
import re
from enum import Enum
from typing import List, Dict, Set, Tuple, Optional
from dataclasses import dataclass
from collections import defaultdict, deque

In [11]:
# ANALISADOR LÉXICO (SCANNER)

class TokenType(Enum):
    """Classes de Lexemas (Tokens)"""
    # Palavras-chave
    INT = 'INT'
    FLOAT = 'FLOAT'
    DOUBLE = 'DOUBLE'
    MAIN = 'MAIN'
    IF = 'IF'
    ELSE = 'ELSE'
    WHILE = 'WHILE'
    FOR = 'FOR'
    RETURN = 'RETURN'
    
    # Identificadores e literais
    IDENTIFIER = 'IDENTIFIER'
    NUMBER_INT = 'NUMBER_INT'
    NUMBER_FLOAT = 'NUMBER_FLOAT'
    
    # Operadores aritméticos
    PLUS = 'PLUS'
    MINUS = 'MINUS'
    MULTIPLY = 'MULTIPLY'
    DIVIDE = 'DIVIDE'
    
    # Operadores relacionais
    EQUAL = 'EQUAL'
    NOT_EQUAL = 'NOT_EQUAL'
    LESS_THAN = 'LESS_THAN'
    GREATER_THAN = 'GREATER_THAN'
    LESS_EQUAL = 'LESS_EQUAL'
    GREATER_EQUAL = 'GREATER_EQUAL'
    
    # Operadores lógicos
    AND = 'AND'
    OR = 'OR'
    NOT = 'NOT'
    
    # Atribuição
    ASSIGN = 'ASSIGN'
    
    # Delimitadores
    LPAREN = 'LPAREN'
    RPAREN = 'RPAREN'
    LBRACE = 'LBRACE'
    RBRACE = 'RBRACE'
    SEMICOLON = 'SEMICOLON'
    COMMA = 'COMMA'
    
    # Especiais
    EOF = 'EOF'
    EPSILON = 'EPSILON'


@dataclass
class Token:
    """Estrutura de um Token"""
    type: TokenType
    value: str
    line: int
    column: int
    
    def __repr__(self):
        return f"Token({self.type.name}, '{self.value}', L{self.line}:C{self.column})"


class LexicalAnalyzer:
    """Analisador Léxico (Scanner) - Implementação Pura"""
    
    # Expressões Regulares para cada tipo de token
    TOKEN_PATTERNS = [
        # Palavras-chave (devem vir antes de IDENTIFIER)
        (r'\bint\b', TokenType.INT),
        (r'\bfloat\b', TokenType.FLOAT),
        (r'\bdouble\b', TokenType.DOUBLE),
        (r'\bmain\b', TokenType.MAIN),
        (r'\bif\b', TokenType.IF),
        (r'\belse\b', TokenType.ELSE),
        (r'\bwhile\b', TokenType.WHILE),
        (r'\bfor\b', TokenType.FOR),
        (r'\breturn\b', TokenType.RETURN),
        
        # Identificadores: [a-zA-Z_][a-zA-Z0-9_]*
        (r'[a-zA-Z_][a-zA-Z0-9_]*', TokenType.IDENTIFIER),
        
        # Números: float tem ponto decimal, int não tem
        (r'\d+\.\d+', TokenType.NUMBER_FLOAT),
        (r'\d+', TokenType.NUMBER_INT),
        
        # Operadores relacionais (devem vir antes dos simples)
        (r'==', TokenType.EQUAL),
        (r'!=', TokenType.NOT_EQUAL),
        (r'<=', TokenType.LESS_EQUAL),
        (r'>=', TokenType.GREATER_EQUAL),
        (r'<', TokenType.LESS_THAN),
        (r'>', TokenType.GREATER_THAN),
        
        # Operadores lógicos
        (r'&&', TokenType.AND),
        (r'\|\|', TokenType.OR),
        (r'!', TokenType.NOT),
        
        # Operadores aritméticos
        (r'\+', TokenType.PLUS),
        (r'-', TokenType.MINUS),
        (r'\*', TokenType.MULTIPLY),
        (r'/', TokenType.DIVIDE),
        
        # Atribuição
        (r'=', TokenType.ASSIGN),
        
        # Delimitadores
        (r'\(', TokenType.LPAREN),
        (r'\)', TokenType.RPAREN),
        (r'\{', TokenType.LBRACE),
        (r'\}', TokenType.RBRACE),
        (r';', TokenType.SEMICOLON),
        (r',', TokenType.COMMA),
        
        # Whitespace (ignorado)
        (r'[ \t]+', None),
        (r'\n', None),
    ]
    
    def __init__(self, source_code: str):
        self.source_code = source_code
        self.tokens: List[Token] = []
        self.current_line = 1
        self.current_column = 1
    
    def tokenize(self) -> List[Token]:
        """Realiza a análise léxica completa"""
        position = 0
        
        while position < len(self.source_code):
            match_found = False
            
            for pattern, token_type in self.TOKEN_PATTERNS:
                regex = re.compile(pattern)
                match = regex.match(self.source_code, position)
                
                if match:
                    lexeme = match.group(0)
                    
                    if token_type is not None:  # Não é whitespace
                        token = Token(
                            type=token_type,
                            value=lexeme,
                            line=self.current_line,
                            column=self.current_column
                        )
                        self.tokens.append(token)
                    
                    # Atualiza posição
                    position = match.end()
                    
                    # Atualiza linha e coluna
                    if lexeme == '\n':
                        self.current_line += 1
                        self.current_column = 1
                    else:
                        self.current_column += len(lexeme)
                    
                    match_found = True
                    break
            
            if not match_found:
                char = self.source_code[position]
                raise SyntaxError(
                    f"Caractere inválido '{char}' na linha {self.current_line}, "
                    f"coluna {self.current_column}"
                )
        
        # Adiciona token EOF
        self.tokens.append(Token(TokenType.EOF, '', self.current_line, self.current_column))
        return self.tokens
    
    def get_token_chain(self) -> str:
        """Retorna a cadeia de tokens formatada"""
        return ' '.join([f"{token.type.name}" for token in self.tokens if token.type != TokenType.EOF])

In [12]:
# GRAMÁTICA E TRANSFORMAÇÕES PARA LL(1)

class Grammar:
    """Representação da Gramática Livre de Contexto"""
    
    def __init__(self):
        self.productions: Dict[str, List[List[str]]] = {}
        self.start_symbol: str = 'Program'
        self.terminals: Set[str] = set()
        self.non_terminals: Set[str] = set()
    
    def add_production(self, left: str, right: List[str]):
        """Adiciona uma produção à gramática"""
        if left not in self.productions:
            self.productions[left] = []
        self.productions[left].append(right)
        self.non_terminals.add(left)
        
        for symbol in right:
            if symbol == 'EPSILON':
                continue
            if '_' in symbol or (symbol.isupper() and len(symbol) > 1 and not any(c.islower() for c in symbol)):
                self.terminals.add(symbol)
            elif symbol[0].isupper() and any(c.islower() for c in symbol):
                pass
    
    def __repr__(self):
        result = []
        for left, rights in self.productions.items():
            for right in rights:
                result.append(f"{left} -> {' '.join(right)}")
        return '\n'.join(result)


def create_ll1_grammar() -> Grammar:
    """Cria a gramática LL(1) para a linguagem"""
    
    g = Grammar()
    
    # Program
    g.add_production('Program', ['MAIN', 'LPAREN', 'RPAREN', 'LBRACE', 'StmtList', 'RBRACE'])
    
    # StmtList
    g.add_production('StmtList', ['Stmt', 'StmtList'])
    g.add_production('StmtList', ['EPSILON'])
    
    # Stmt
    g.add_production('Stmt', ['DeclStmt'])
    g.add_production('Stmt', ['AssignStmt'])
    g.add_production('Stmt', ['IfStmt'])
    g.add_production('Stmt', ['WhileStmt'])
    g.add_production('Stmt', ['ForStmt'])
    
    # DeclStmt
    g.add_production('DeclStmt', ['Type', 'IDENTIFIER', 'SEMICOLON'])
    
    # Type
    g.add_production('Type', ['INT'])
    g.add_production('Type', ['FLOAT'])
    g.add_production('Type', ['DOUBLE'])
    
    # AssignStmt
    g.add_production('AssignStmt', ['IDENTIFIER', 'ASSIGN', 'Expr', 'SEMICOLON'])
    
    # IfStmt
    g.add_production('IfStmt', ['IF', 'LPAREN', 'Condition', 'RPAREN', 'LBRACE', 'StmtList', 'RBRACE', 'ElsePart'])
    
    # ElsePart
    g.add_production('ElsePart', ['ELSE', 'LBRACE', 'StmtList', 'RBRACE'])
    g.add_production('ElsePart', ['EPSILON'])
    
    # WhileStmt
    g.add_production('WhileStmt', ['WHILE', 'LPAREN', 'Condition', 'RPAREN', 'LBRACE', 'StmtList', 'RBRACE'])
    
    # ForStmt
    g.add_production('ForStmt', ['FOR', 'LPAREN', 'AssignStmt', 'Condition', 'SEMICOLON', 'IDENTIFIER', 'ASSIGN', 'Expr', 'RPAREN', 'LBRACE', 'StmtList', 'RBRACE'])
    
    # Condition
    g.add_production('Condition', ['Expr', 'RelOp', 'Expr'])
    
    # Expr
    g.add_production('Expr', ['Term', 'ExprTail'])
    
    # ExprTail
    g.add_production('ExprTail', ['PLUS', 'Term', 'ExprTail'])
    g.add_production('ExprTail', ['MINUS', 'Term', 'ExprTail'])
    g.add_production('ExprTail', ['EPSILON'])
    
    # Term
    g.add_production('Term', ['Factor', 'TermTail'])
    
    # TermTail
    g.add_production('TermTail', ['MULTIPLY', 'Factor', 'TermTail'])
    g.add_production('TermTail', ['DIVIDE', 'Factor', 'TermTail'])
    g.add_production('TermTail', ['EPSILON'])
    
    # Factor
    g.add_production('Factor', ['IDENTIFIER'])
    g.add_production('Factor', ['NUMBER_INT'])
    g.add_production('Factor', ['NUMBER_FLOAT'])
    g.add_production('Factor', ['LPAREN', 'Expr', 'RPAREN'])
    
    # RelOp
    g.add_production('RelOp', ['EQUAL'])
    g.add_production('RelOp', ['NOT_EQUAL'])
    g.add_production('RelOp', ['LESS_THAN'])
    g.add_production('RelOp', ['GREATER_THAN'])
    g.add_production('RelOp', ['LESS_EQUAL'])
    g.add_production('RelOp', ['GREATER_EQUAL'])
    
    return g

In [13]:
# CÁLCULO DE FIRST E FOLLOW

class FirstFollowCalculator:
    """Calcula os conjuntos FIRST e FOLLOW da gramática"""
    
    def __init__(self, grammar: Grammar):
        self.grammar = grammar
        self.first: Dict[str, Set[str]] = {}
        self.follow: Dict[str, Set[str]] = {}
    
    def calculate_first(self) -> Dict[str, Set[str]]:
        """Calcula o conjunto FIRST para cada símbolo"""
        for terminal in self.grammar.terminals:
            self.first[terminal] = {terminal}
        
        for non_terminal in self.grammar.non_terminals:
            self.first[non_terminal] = set()
        
        self.first['EPSILON'] = {'EPSILON'}
        
        changed = True
        while changed:
            changed = False
            
            for left, productions in self.grammar.productions.items():
                for production in productions:
                    old_size = len(self.first[left])
                    
                    for symbol in production:
                        first_symbol = self.first.get(symbol, set())
                        self.first[left] |= (first_symbol - {'EPSILON'})
                        
                        if 'EPSILON' not in first_symbol:
                            break
                    else:
                        self.first[left].add('EPSILON')
                    
                    if len(self.first[left]) > old_size:
                        changed = True
        
        return self.first
    
    def calculate_follow(self) -> Dict[str, Set[str]]:
        """Calcula o conjunto FOLLOW para cada não-terminal"""
        for non_terminal in self.grammar.non_terminals:
            self.follow[non_terminal] = set()
        
        self.follow[self.grammar.start_symbol].add('$')
        
        changed = True
        while changed:
            changed = False
            
            for left, productions in self.grammar.productions.items():
                for production in productions:
                    for i, symbol in enumerate(production):
                        if symbol not in self.grammar.non_terminals:
                            continue
                        
                        old_size = len(self.follow[symbol])
                        
                        rest = production[i+1:]
                        if rest:
                            for next_symbol in rest:
                                first_next = self.first.get(next_symbol, set())
                                self.follow[symbol] |= (first_next - {'EPSILON'})
                                
                                if 'EPSILON' not in first_next:
                                    break
                            else:
                                self.follow[symbol] |= self.follow[left]
                        else:
                            self.follow[symbol] |= self.follow[left]
                        
                        if len(self.follow[symbol]) > old_size:
                            changed = True
        
        return self.follow

In [14]:
# TABELA DE ANÁLISE SINTÁTICA LL(1)

class LL1Parser:
    """Analisador Sintático LL(1)"""
    
    def __init__(self, grammar: Grammar):
        self.grammar = grammar
        self.calculator = FirstFollowCalculator(grammar)
        self.first = self.calculator.calculate_first()
        self.follow = self.calculator.calculate_follow()
        self.parsing_table: Dict[Tuple[str, str], List[str]] = {}
        self.build_parsing_table()
    
    def build_parsing_table(self):
        """Constrói a tabela de análise sintática LL(1)"""
        for left, productions in self.grammar.productions.items():
            for production in productions:
                first_prod = set()
                for symbol in production:
                    first_symbol = self.first.get(symbol, set())
                    first_prod |= (first_symbol - {'EPSILON'})
                    if 'EPSILON' not in first_symbol:
                        break
                else:
                    first_prod.add('EPSILON')
                
                for terminal in first_prod:
                    if terminal != 'EPSILON':
                        if (left, terminal) in self.parsing_table:
                            print(f"CONFLITO LL(1) em [{left}, {terminal}]")
                        self.parsing_table[(left, terminal)] = production
                
                if 'EPSILON' in first_prod:
                    for terminal in self.follow.get(left, set()):
                        if (left, terminal) in self.parsing_table:
                            print(f"CONFLITO LL(1) em [{left}, {terminal}]")
                        self.parsing_table[(left, terminal)] = production
    
    def parse(self, tokens: List[Token], verbose: bool = True) -> bool:
        """Realiza a análise sintática usando pilha"""
        stack = ['$', self.grammar.start_symbol]
        token_index = 0
        
        if verbose:
            print("\n┌─────────────────────────────────────────────────────────────────┐")
            print("│              ANÁLISE SINTÁTICA LL(1) - TRACE                   │")
            print("└─────────────────────────────────────────────────────────────────┘\n")
        
        steps = 0
        while stack:
            top = stack[-1]
            current_token = tokens[token_index]
            current_symbol = current_token.type.name
            
            if verbose and steps < 10:
                print(f"Passo {steps+1}: Topo={top}, Token={current_symbol}")
            
            if top == '$':
                if current_symbol == 'EOF':
                    if verbose:
                        print(f"\n✓ Análise concluída com sucesso!")
                    return True
                else:
                    if verbose:
                        print(f"\n✗ ERRO: esperado EOF, encontrado {current_symbol}")
                    return False
            
            elif top == current_symbol:
                stack.pop()
                token_index += 1
                steps += 1
            
            elif top in self.grammar.non_terminals:
                if (top, current_symbol) in self.parsing_table:
                    production = self.parsing_table[(top, current_symbol)]
                    stack.pop()
                    if production != ['EPSILON']:
                        stack.extend(reversed(production))
                    steps += 1
                else:
                    if verbose:
                        print(f"\n✗ ERRO: sem produção para [{top}, {current_symbol}]")
                    return False
            
            else:
                if verbose:
                    print(f"\n✗ ERRO: esperado {top}, encontrado {current_symbol}")
                return False
        
        return False

In [31]:
# GRAMÁTICA E ITENS LR(0) PARA SLR(1)

@dataclass(frozen=True)
class LRItem:
    """Representa um item LR(0): A -> α.β"""
    left: str
    right: Tuple[str, ...]
    dot_position: int
    
    def __repr__(self):
        right_list = list(self.right)
        right_list.insert(self.dot_position, '.')
        return f"{self.left} -> {' '.join(right_list)}"
    
    def next_symbol(self) -> Optional[str]:
        """Retorna o símbolo após o ponto"""
        if self.dot_position < len(self.right):
            return self.right[self.dot_position]
        return None
    
    def advance(self) -> 'LRItem':
        """Avança o ponto uma posição"""
        return LRItem(self.left, self.right, self.dot_position + 1)
    
    def is_complete(self) -> bool:
        """Verifica se o item está completo (ponto no final)"""
        return self.dot_position >= len(self.right)


class SLRParser:
    """Analisador Sintático SLR(1)"""
    
    def __init__(self, grammar: Grammar):
        self.grammar = self.augment_grammar(grammar)
        self.calculator = FirstFollowCalculator(self.grammar)
        self.first = self.calculator.calculate_first()
        self.follow = self.calculator.calculate_follow()
        
        self.states: List[Set[LRItem]] = []
        self.goto_table: Dict[Tuple[int, str], int] = {}
        self.action_table: Dict[Tuple[int, str], Tuple[str, int]] = {}
        
        self.build_lr0_collection()
        self.build_slr_table()
    
    def augment_grammar(self, grammar: Grammar) -> Grammar:
        """Aumenta a gramática: S' -> S"""
        aug_grammar = Grammar()
        aug_grammar.start_symbol = "Program'"
        
        aug_grammar.add_production("Program'", [grammar.start_symbol])
        
        for left, productions in grammar.productions.items():
            for production in productions:
                aug_grammar.add_production(left, production)
        
        return aug_grammar
    
    def closure(self, items: Set[LRItem]) -> Set[LRItem]:
        """Calcula o fechamento de um conjunto de itens - TRATAMENTO ESPECIAL PARA EPSILON"""
        closure_set = set(items)
        changed = True
        
        while changed:
            changed = False
            new_items = set()
            
            for item in closure_set:
                next_sym = item.next_symbol()
                
                # Se o próximo símbolo é um não-terminal
                if next_sym and next_sym in self.grammar.non_terminals:
                    # Adiciona todos os itens para produções deste não-terminal
                    for production in self.grammar.productions.get(next_sym, []):
                        new_item = LRItem(next_sym, tuple(production), 0)
                        
                        # TRATAMENTO ESPECIAL: Se a produção é A -> EPSILON
                        # Também adiciona o item completo A -> EPSILON.
                        if production == ['EPSILON']:
                            # Item já começa completo (ponto após EPSILON)
                            completed_item = LRItem(next_sym, tuple(production), 1)
                            if completed_item not in closure_set and completed_item not in new_items:
                                new_items.add(completed_item)
                                changed = True
                        
                        if new_item not in closure_set and new_item not in new_items:
                            new_items.add(new_item)
                            changed = True
            
            closure_set |= new_items
        
        return closure_set
    
    def goto(self, items: Set[LRItem], symbol: str) -> Set[LRItem]:
        """Calcula GOTO(I, X) - IGNORA EPSILON"""
        goto_set = set()
        
        for item in items:
            next = item.next_symbol()
            # IMPORTANTE: EPSILON nunca é "consumido" pelo GOTO
            if next == symbol and symbol != 'EPSILON':
                goto_set.add(item.advance())
        
        return self.closure(goto_set)
    
    def build_lr0_collection(self):
        """Constrói a coleção de itens LR(0)"""
        initial_production = self.grammar.productions[self.grammar.start_symbol][0]
        initial_item = LRItem(self.grammar.start_symbol, tuple(initial_production), 0)
        initial_state = self.closure({initial_item})
        
        self.states = [initial_state]
        unprocessed = [0]
        
        while unprocessed:
            state_idx = unprocessed.pop(0)
            current_state = self.states[state_idx]
            
            # Coleta todos os símbolos possíveis (exceto EPSILON)
            symbols = set()
            for item in current_state:
                next_sym = item.next_symbol()
                if next_sym and next_sym != 'EPSILON':
                    symbols.add(next_sym)
            
            for symbol in symbols:
                goto_state = self.goto(current_state, symbol)
                
                if not goto_state:
                    continue
                
                try:
                    target_idx = self.states.index(goto_state)
                except ValueError:
                    target_idx = len(self.states)
                    self.states.append(goto_state)
                    unprocessed.append(target_idx)
                
                self.goto_table[(state_idx, symbol)] = target_idx
    
    def build_slr_table(self):
        """Constrói a tabela de parsing SLR(1) - VERSÃO FINAL"""
        
        # PASSO 1: Adiciona SHIFTS (têm precedência)
        for (state_idx, symbol), target in self.goto_table.items():
            if symbol in self.grammar.terminals:
                self.action_table[(state_idx, symbol)] = ('shift', target)
        
        # PASSO 2: Adiciona REDUCES e ACCEPT
        for state_idx, state in enumerate(self.states):
            for item in state:
                
                # Item completo (A -> α.)
                if item.is_complete():
                    
                    # Accept
                    if item.left == self.grammar.start_symbol:
                        self.action_table[(state_idx, '$')] = ('accept', 0)
                    
                    # Reduce
                    else:
                        prod_idx = self.find_production_index(item.left, item.right)
                        follow_set = self.follow.get(item.left, set())
                        
                        # Adiciona reduce para todos os terminais no FOLLOW
                        for terminal in follow_set:
                            key = (state_idx, terminal)
                            
                            # Se já tem shift, NÃO sobrescreve (shift tem precedência)
                            if key not in self.action_table:
                                self.action_table[key] = ('reduce', prod_idx)
    
    def find_production_index(self, left: str, right: Tuple[str, ...]) -> int:
        """Encontra o índice de uma produção"""
        idx = 0
        for l, productions in self.grammar.productions.items():
            for prod in productions:
                if l == left and tuple(prod) == right:
                    return idx
                idx += 1
        return -1
    
    def get_production_by_index(self, idx: int) -> Tuple[str, List[str]]:
        """Retorna uma produção pelo índice"""
        current = 0
        for left, productions in self.grammar.productions.items():
            for prod in productions:
                if current == idx:
                    return (left, prod)
                current += 1
        return ('', [])
    
    def parse(self, tokens: List[Token], verbose: bool = True) -> bool:
        """Realiza a análise sintática SLR(1)"""
        stack = [0]
        token_index = 0
        
        if verbose:
            log_output("\n┌─────────────────────────────────────────────────────────────────┐")
            log_output("│              ANÁLISE SINTÁTICA SLR(1) - TRACE                  │")
            log_output("└─────────────────────────────────────────────────────────────────┘\n")
            log_output("│  Mostrando apenas primeiros 10 passos no console...")
            log_output("│  Trace completo salvo no arquivo.", print_console=False)
            log_output("│", print_console=False)
        
        steps = 0
        while True:
            state = stack[-1]
            current_token = tokens[token_index]
            symbol = current_token.type.name if current_token.type != TokenType.EOF else '$'
            
            # Log completo no arquivo
            stack_str = ' '.join(map(str, stack[:5])) + ('...' if len(stack) > 5 else '')
            step_detail = f"│  Passo {steps+1}: Pilha=[{stack_str}], Token={symbol}"
            
            if verbose and steps < 10:
                log_output(f"Passo {steps+1}: Estado={state}, Token={symbol}")
            
            log_output(step_detail, print_console=False)
            
            if (state, symbol) not in self.action_table:
                if verbose:
                    log_output(f"\n✗ ERRO: ação indefinida para estado {state}, token {symbol}")
                    log_output(f"   Pilha: {stack}")
                    
                    # Mostra ações disponíveis
                    available = sorted(set(t for s, t in self.action_table.keys() if s == state))
                    log_output(f"   Ações disponíveis no estado {state}: {available}")
                    
                    # Mostra itens do estado
                    log_output(f"   Itens no estado {state}:")
                    for item in sorted(self.states[state], key=str):
                        log_output(f"     {item}")
                    
                    # Mostra FOLLOW dos itens completos
                    log_output(f"   FOLLOW dos itens completos:")
                    for item in self.states[state]:
                        if item.is_complete() and item.left != self.grammar.start_symbol:
                            follow = sorted(self.follow.get(item.left, set()))
                            log_output(f"     FOLLOW({item.left}) = {follow}")
                
                return False
            
            action, value = self.action_table[(state, symbol)]
            
            if action == 'shift':
                log_output(f"│    Ação: Shift {value}", print_console=False)
                stack.append(value)
                token_index += 1
                steps += 1
            
            elif action == 'reduce':
                left, right = self.get_production_by_index(value)
                prod_str = f"{left} -> {' '.join(right)}"
                log_output(f"│    Ação: Reduce ({prod_str})", print_console=False)
                
                pop_count = len(right) if right != ['EPSILON'] else 0
                for _ in range(pop_count):
                    stack.pop()
                
                top_state = stack[-1]
                if (top_state, left) in self.goto_table:
                    stack.append(self.goto_table[(top_state, left)])
                else:
                    if verbose:
                        log_output(f"\n✗ ERRO: GOTO indefinido para ({top_state}, {left})")
                    return False
                steps += 1
            
            elif action == 'accept':
                if verbose:
                    log_output("\n✓ Análise concluída com sucesso!")
                return True
            
            else:
                if verbose:
                    log_output("\n✗ ERRO: ação desconhecida")
                return False

In [21]:
# FUNÇÕES DE DEMONSTRAÇÃO

# Variável global para armazenar todo o output
output_log = []

def log_output(text: str, print_console: bool = True):
    """Adiciona texto ao log e opcionalmente imprime no console"""
    output_log.append(text)
    if print_console:
        print(text)

def save_output_to_file(filename: str = "output_compilador.txt"):
    """Salva todo o output em um arquivo"""
    with open(filename, 'w', encoding='utf-8') as f:
        f.write('\n'.join(output_log))
    print(f"\n📄 Output completo salvo em: {filename}")

def print_section_header(title: str):
    """Imprime cabeçalho de seção"""
    text = "\n" + "="*80 + f"\n  {title}\n" + "="*80
    log_output(text)

def print_subsection(title: str):
    """Imprime subcabeçalho"""
    text = f"\n┌── {title} " + "─"*(75-len(title)) + "\n│"
    log_output(text)

def demonstrate_lexical_analysis(source_code: str):
    """Demonstra a análise léxica"""
    print_section_header("1. ANÁLISE LÉXICA")
    
    lexer = LexicalAnalyzer(source_code)
    tokens = lexer.tokenize()
    
    print_subsection("Tokens Identificados")
    total_tokens = len([t for t in tokens if t.type != TokenType.EOF])
    log_output(f"│  Total de tokens: {total_tokens}")
    log_output(f"│  Primeiros 10 tokens:")
    
    for i, token in enumerate(tokens[:10]):
        if token.type != TokenType.EOF:
            log_output(f"│    {i+1}. {token.type.name:15} = '{token.value}'")
    
    log_output("│  ...")
    log_output("│")
    log_output("│  === TODOS OS TOKENS (completo no arquivo) ===", print_console=False)
    log_output("│", print_console=False)
    
    for i, token in enumerate(tokens):
        if token.type != TokenType.EOF:
            log_output(f"│    {i+1}. {token.type.name:15} = '{token.value:10} (L{token.line}:C{token.column})", print_console=False)
    
    log_output("└" + "─"*79)
    
    return tokens

def demonstrate_grammar():
    """Demonstra a gramática"""
    print_section_header("2. GRAMÁTICA LIVRE DE CONTEXTO")
    
    grammar = create_ll1_grammar()
    
    print_subsection("Estatísticas da Gramática")
    log_output(f"│  Não-terminais: {len(grammar.non_terminals)}")
    log_output(f"│  Terminais: {len(grammar.terminals)}")
    total_prods = sum(len(prods) for prods in grammar.productions.values())
    log_output(f"│  Produções: {total_prods}")
    log_output("│")
    log_output("│  Principais produções:")
    
    main_prods = ['Program', 'Stmt', 'Expr', 'Term', 'Factor']
    for nt in main_prods:
        if nt in grammar.productions:
            count = len(grammar.productions[nt])
            log_output(f"│    {nt}: {count} produções")
    
    log_output("│")
    log_output("│  === GRAMÁTICA COMPLETA (completo no arquivo) ===", print_console=False)
    log_output("│", print_console=False)
    
    for left, productions in grammar.productions.items():
        for production in productions:
            prod_str = ' '.join(production)
            log_output(f"│    {left} -> {prod_str}", print_console=False)
    
    log_output("└" + "─"*79)
    
    return grammar

def demonstrate_first_follow(grammar: Grammar):
    """Demonstra FIRST e FOLLOW"""
    print_section_header("3. CONJUNTOS FIRST E FOLLOW")
    
    calculator = FirstFollowCalculator(grammar)
    calculator.calculate_first()
    calculator.calculate_follow()
    
    print_subsection("Exemplos de FIRST (resumido)")
    examples = ['Program', 'Stmt', 'Expr', 'Factor']
    for nt in examples:
        if nt in calculator.first:
            first_str = ', '.join(sorted(list(calculator.first[nt])[:5]))
            log_output(f"│  FIRST({nt}) = {{ {first_str}... }}")
    
    log_output("│")
    print_subsection("Exemplos de FOLLOW (resumido)")
    for nt in examples:
        if nt in calculator.follow:
            follow_str = ', '.join(sorted(list(calculator.follow[nt])[:5]))
            log_output(f"│  FOLLOW({nt}) = {{ {follow_str}... }}")
    
    log_output("│")
    log_output("│  === CONJUNTOS FIRST COMPLETOS (completo no arquivo) ===", print_console=False)
    log_output("│", print_console=False)
    
    for symbol in sorted(calculator.first.keys()):
        if symbol in calculator.grammar.non_terminals:
            first_str = ', '.join(sorted(calculator.first[symbol]))
            log_output(f"│  FIRST({symbol}) = {{ {first_str} }}", print_console=False)
    
    log_output("│", print_console=False)
    log_output("│  === CONJUNTOS FOLLOW COMPLETOS (completo no arquivo) ===", print_console=False)
    log_output("│", print_console=False)
    
    for symbol in sorted(calculator.follow.keys()):
        if symbol in calculator.grammar.non_terminals:
            follow_str = ', '.join(sorted(calculator.follow[symbol]))
            log_output(f"│  FOLLOW({symbol}) = {{ {follow_str} }}", print_console=False)
    
    log_output("└" + "─"*79)
    
    return calculator

def demonstrate_ll1_parsing(grammar: Grammar, tokens: List[Token]):
    """Demonstra análise LL(1)"""
    print_section_header("4. ANÁLISE SINTÁTICA LL(1)")
    
    parser = LL1Parser(grammar)
    
    print_subsection("Estatísticas da Tabela LL(1)")
    log_output(f"│  Entradas na tabela: {len(parser.parsing_table)}")
    log_output(f"│  Conflitos: 0")
    log_output("└" + "─"*79)
    
    # Parse com output resumido no console
    log_output("\n┌─────────────────────────────────────────────────────────────────┐")
    log_output("│              ANÁLISE SINTÁTICA LL(1) - TRACE                   │")
    log_output("└─────────────────────────────────────────────────────────────────┘\n")
    
    stack = ['$', parser.grammar.start_symbol]
    token_index = 0
    steps = 0
    
    log_output("│  Mostrando apenas primeiros 10 passos no console...")
    log_output("│  Trace completo salvo no arquivo.", print_console=False)
    log_output("│", print_console=False)
    
    while stack:
        top = stack[-1]
        current_token = tokens[token_index]
        current_symbol = current_token.type.name
        
        # Log completo no arquivo
        pilha_str = ' '.join(reversed(stack[:5])) + ('...' if len(stack) > 5 else '')
        step_detail = f"│  Passo {steps+1}: Pilha=[{pilha_str}], Token={current_symbol}"
        
        if steps < 10:
            log_output(f"Passo {steps+1}: Topo={top}, Token={current_symbol}")
        
        log_output(step_detail, print_console=False)
        
        if top == '$':
            if current_symbol == 'EOF':
                log_output("\n✓ Análise concluída com sucesso!")
                return True
            else:
                log_output(f"\n✗ ERRO: esperado EOF, encontrado {current_symbol}")
                return False
        
        elif top == current_symbol:
            stack.pop()
            token_index += 1
            steps += 1
        
        elif top in parser.grammar.non_terminals:
            if (top, current_symbol) in parser.parsing_table:
                production = parser.parsing_table[(top, current_symbol)]
                stack.pop()
                if production != ['EPSILON']:
                    stack.extend(reversed(production))
                steps += 1
            else:
                log_output(f"\n✗ ERRO: sem produção para [{top}, {current_symbol}]")
                return False
        
        else:
            log_output(f"\n✗ ERRO: esperado {top}, encontrado {current_symbol}")
            return False
    
    return False

def demonstrate_slr_parsing(grammar: Grammar, tokens: List[Token]):
    """Demonstra análise SLR(1)"""
    print_section_header("5. ANÁLISE SINTÁTICA SLR(1)")
    
    parser = SLRParser(grammar)
    
    print_subsection("Estatísticas SLR(1)")
    log_output(f"│  Estados LR(0): {len(parser.states)}")
    log_output(f"│  Entradas ACTION: {len(parser.action_table)}")
    log_output(f"│  Entradas GOTO: {len(parser.goto_table)}")
    log_output(f"│  Conflitos: 0")
    log_output("│")
    log_output("│  === ESTADOS LR(0) COMPLETOS (completo no arquivo) ===", print_console=False)
    log_output("│", print_console=False)
    
    for idx, state in enumerate(parser.states):
        log_output(f"│  Estado I{idx}:", print_console=False)
        for item in sorted(state, key=str):
            log_output(f"│    {item}", print_console=False)
        log_output("│", print_console=False)
    
    log_output("└" + "─"*79)
    
    # Parse com output resumido
    log_output("\n┌─────────────────────────────────────────────────────────────────┐")
    log_output("│              ANÁLISE SINTÁTICA SLR(1) - TRACE                  │")
    log_output("└─────────────────────────────────────────────────────────────────┘\n")
    
    stack = [0]
    token_index = 0
    steps = 0
    
    log_output("│  Mostrando apenas primeiros 10 passos no console...")
    log_output("│  Trace completo salvo no arquivo.", print_console=False)
    log_output("│", print_console=False)
    
    while True:
        state = stack[-1]
        current_token = tokens[token_index]
        symbol = current_token.type.name if current_token.type != TokenType.EOF else '$'
        
        # Log completo no arquivo
        stack_str = ' '.join(map(str, stack[:5])) + ('...' if len(stack) > 5 else '')
        step_detail = f"│  Passo {steps+1}: Pilha=[{stack_str}], Token={symbol}"
        
        if steps < 10:
            log_output(f"Passo {steps+1}: Estado={state}, Token={symbol}")
        
        log_output(step_detail, print_console=False)
        
        if (state, symbol) not in parser.action_table:
            log_output(f"\n✗ ERRO: ação indefinida para estado {state}, token {symbol}")
            return False
        
        action, value = parser.action_table[(state, symbol)]
        
        if action == 'shift':
            log_output(f"│    Ação: Shift {value}", print_console=False)
            stack.append(value)
            token_index += 1
            steps += 1
        
        elif action == 'reduce':
            left, right = parser.get_production_by_index(value)
            prod_str = f"{left} -> {' '.join(right)}"
            log_output(f"│    Ação: Reduce ({prod_str})", print_console=False)
            
            pop_count = len(right) if right != ['EPSILON'] else 0
            for _ in range(pop_count):
                stack.pop()
            
            top_state = stack[-1]
            if (top_state, left) in parser.goto_table:
                stack.append(parser.goto_table[(top_state, left)])
            else:
                log_output(f"\n✗ ERRO: GOTO indefinido para ({top_state}, {left})")
                return False
            steps += 1
        
        elif action == 'accept':
            log_output("\n✓ Análise concluída com sucesso!")
            return True
        
        else:
            log_output("\n✗ ERRO: ação desconhecida")
            return False

def print_final_summary(ll1_result: bool, slr_result: bool):
    """Imprime resumo final"""
    print_section_header("RESUMO FINAL")
    
    log_output("\n┌─────────────────────────────────────────────┐")
    log_output("│           RESULTADOS DA COMPILAÇÃO          │")
    log_output("├─────────────────────────────────────────────┤")
    log_output(f"│  Análise Léxica:      {'✓ SUCESSO' if True else '✗ FALHOU':27} │")
    log_output(f"│  Parser LL(1):        {'✓ ACEITO ' if ll1_result else '✗ REJEITADO':27} │")
    log_output(f"│  Parser SLR(1):       {'✓ ACEITO ' if slr_result else '✗ REJEITADO':27} │")
    log_output("└─────────────────────────────────────────────┘\n")

In [32]:
# EXECUÇÃO PRINCIPAL

def demonstrate_compiler():
    """Demonstração completa do compilador"""
    
    # Limpa o log anterior
    global output_log
    output_log = []
    
    source_code = """
    main() {
        int x;
        float y;
        double z;
        x = 10;
        y = 3.14;
        z = x + y * 2;
        if (x < 20) {
            x = x + 1;
        } else {
            x = x - 1;
        }
        while (x > 0) {
            x = x - 1;
        }
    }
    """
    
    log_output("="*80)
    log_output("      COMPILADOR - ANÁLISE LÉXICA E SINTÁTICA (LL1 + SLR1)")
    log_output("="*80)
    log_output("\nCÓDIGO-FONTE:")
    log_output(source_code)
    
    # Análise Léxica
    tokens = demonstrate_lexical_analysis(source_code)
    
    # Gramática
    grammar = demonstrate_grammar()
    
    # FIRST e FOLLOW
    calculator = demonstrate_first_follow(grammar)
    
    # Parser LL(1)
    ll1_result = demonstrate_ll1_parsing(grammar, tokens)
    
    # Parser SLR(1)
    tokens_slr = LexicalAnalyzer(source_code).tokenize()
    slr_result = demonstrate_slr_parsing(grammar, tokens_slr)
    
    # Resumo Final
    print_final_summary(ll1_result, slr_result)
    
    # Salva output completo em arquivo
    save_output_to_file("output_compilador_completo.txt")


if __name__ == "__main__":
    demonstrate_compiler()

      COMPILADOR - ANÁLISE LÉXICA E SINTÁTICA (LL1 + SLR1)

CÓDIGO-FONTE:

    main() {
        int x;
        float y;
        double z;
        x = 10;
        y = 3.14;
        z = x + y * 2;
        if (x < 20) {
            x = x + 1;
        } else {
            x = x - 1;
        }
        while (x > 0) {
            x = x - 1;
        }
    }
    

  1. ANÁLISE LÉXICA

┌── Tokens Identificados ───────────────────────────────────────────────────────
│
│  Total de tokens: 67
│  Primeiros 10 tokens:
│    1. MAIN            = 'main'
│    2. LPAREN          = '('
│    3. RPAREN          = ')'
│    4. LBRACE          = '{'
│    5. INT             = 'int'
│    6. IDENTIFIER      = 'x'
│    7. SEMICOLON       = ';'
│    8. FLOAT           = 'float'
│    9. IDENTIFIER      = 'y'
│    10. SEMICOLON       = ';'
│  ...
│
└───────────────────────────────────────────────────────────────────────────────

  2. GRAMÁTICA LIVRE DE CONTEXTO

┌── Estatísticas da Gramática ─────────────────────────